# Unit 6. From text to speech

Bir önceki ünitede, konuşmayı metne dönüştürmek için Transformers'ı nasıl kullanacağınızı öğrendiniz. Şimdi senaryoyu tersine çevirelim ve verilen bir giriş metnini insan konuşmasına benzeyen bir ses çıkışına nasıl dönüştürebileceğinizi görelim.

Bu ünitede inceleyeceğimiz görev "Text-to-speech" (TTS) olarak adlandırılmaktadır. Metni duyulabilir insan konuşmasına dönüştürebilen modeller çok çeşitli potansiyel uygulamalara sahiptir:

- Yardımcı uygulamalar: Görme engelli kişilerin dijital içeriğe ses aracılığıyla erişmesini sağlamak için bu modellerden yararlanabilecek araçları düşünün.
- Sesli kitap anlatımı: yazılı kitapların sesli forma dönüştürülmesi, edebiyatı dinlemeyi tercih eden veya okuma güçlüğü çeken bireyler için daha erişilebilir hale getirir.
- Sanal asistanlar: TTS modelleri Siri, Google Assistant veya Amazon Alexa gibi sanal asistanların temel bir bileşenidir. Uyandırma kelimesini yakalamak için bir sınıflandırma modeli kullandıktan ve isteğinizi işlemek için ASR modelini kullandıktan sonra, sorunuza yanıt vermek için bir TTS modeli kullanabilirler.
- Eğlence, oyun ve dil öğrenimi: NPC karakterlerinize ses verin, oyun etkinliklerini anlatın veya dil öğrenenlere doğru telaffuz ve kelime ve kelime öbeklerinin tonlama örnekleriyle yardımcı olun.

Bunlar sadece birkaç örnek ve eminim daha fazlasını hayal edebilirsiniz! Ancak bu kadar güç beraberinde sorumluluğu da getirir ve TTS modellerinin kötü niyetli amaçlar için kullanılma potansiyeline sahip olduğunu vurgulamak önemlidir. Örneğin, yeterli ses örnekleriyle, kötü niyetli aktörler potansiyel olarak ikna edici sahte ses kayıtları oluşturabilir ve bu da birinin sesinin dolandırıcılık veya manipülasyon amacıyla yetkisiz kullanımına yol açabilir. Kendi sistemlerinize ince ayar yapmak için veri toplamayı planlıyorsanız, gizlilik ve bilgilendirilmiş onayı dikkatlice değerlendirin. Ses verileri, seslerinin bir TTS sisteminde kullanılmasının amacını, kapsamını ve potansiyel risklerini anladıklarından emin olmak için bireylerin açık rızası ile elde edilmelidir. Lütfen metinden sese özelliğini sorumlu bir şekilde kullanın.

Neler öğreneceksiniz ve neler inşa edeceksiniz Bu ünitede şunlardan bahsedeceğiz:

- Metinden konuşmaya eğitim için uygun veri kümeleri 
- Metinden konuşmaya için önceden eğitilmiş modeller 
- SpeechT5'e yeni bir dilde ince ayar yapma 
- TTS modellerini değerlendirme 

# **Text-to-speech datasets**

Metinden konuşmaya görevi (konuşma sentezi olarak da adlandırılır) bir dizi zorlukla birlikte gelir.

İlk olarak, daha önce tartışılan otomatik konuşma tanımada olduğu gibi, metin ve konuşma arasındaki hizalama zor olabilir. Bununla birlikte, ASR'den farklı olarak, TTS one-to-many problemidir, yani aynı metin birçok farklı şekilde sentezlenebilir. Günlük olarak duyduğunuz konuşmalardaki seslerin ve konuşma tarzlarının çeşitliliğini düşünün - her kişinin aynı cümleyi söylemenin farklı bir yolu vardır, ancak hepsi geçerli ve doğrudur! Farklı çıktılar (spektrogramlar veya ses dalga formları) bile aynı temel gerçeğe karşılık gelebilir. Modelin her bir fonem, kelime veya cümle için doğru süre ve zamanlamayı üretmeyi öğrenmesi gerekir ki bu da özellikle uzun ve karmaşık cümleler için zorlayıcı olabilir.

Bir de uzun mesafeli bağımlılık sorunu var: dilin zamansal bir yönü var ve bir cümlenin anlamını anlamak genellikle çevredeki kelimelerin bağlamını dikkate almayı gerektiriyor. TTS modelinin uzun diziler boyunca bağlamsal bilgileri yakalamasını ve korumasını sağlamak, tutarlı ve kulağa doğal gelen konuşma üretmek için çok önemlidir.

Son olarak, TTS modellerinin eğitimi genellikle **metin ve ilgili konuşma kayıtlarının çiftlerini** gerektirir. Bunun da ötesinde, modelin çeşitli konuşmacılar ve konuşma stilleri için doğal ses veren konuşma üretebilmesini sağlamak için, verilerin birden fazla konuşmacıdan çeşitli ve temsili konuşma örnekleri içermesi gerekir. Bu tür verileri toplamak pahalı, zaman alıcı ve bazı diller için mümkün değildir. Neden **ASR (otomatik konuşma tanıma) için tasarlanmış bir veri kümesini alıp bir TTS modelini eğitmek için kullanmayalım diye düşünebilirsiniz**. Ne yazık ki, **otomatik konuşma tanıma (ASR) veri kümeleri en iyi seçenek değildir. Aşırı arka plan gürültüsü gibi ASR için faydalı olan özellikler, TTS'de genellikle istenmeyen özelliklerdir**. Gürültülü bir sokak kaydından konuşmayı seçebilmek harikadır, ancak sesli asistanınız size arka planda arabalar korna çalarken ve inşaat tam gaz devam ederken yanıt verirse o kadar da iyi değildir. Yine de, bazı ASR veri kümeleri bazen ince ayar yapmak için yararlı olabilir, çünkü en kaliteli, çok dilli ve çok konuşmacılı TTS veri kümelerini bulmak oldukça zor olabilir.

Şimdi Hub'da bulabileceğiniz TTS için uygun birkaç veri kümesini inceleyelim.

## LJSpeech

LJSpeech, ilgili transkripsiyonlarıyla eşleştirilmiş 13.100 İngilizce ses klibinden oluşan bir veri kümesidir. Veri kümesi, **tek bir konuşmacının** İngilizce 7 kurgusal olmayan kitaptan cümleler okuduğu kayıtları içerir. LJSpeech, yüksek ses kalitesi ve çeşitli dilsel içeriği nedeniyle TTS modellerini değerlendirmek için genellikle bir ölçüt olarak kullanılır.

## Multilingual LibriSpeech

Çok Dilli LibriSpeech, İngilizce sesli kitapların okunduğu büyük ölçekli bir koleksiyon olan LibriSpeech veri setinin çok dilli bir uzantısıdır. Çok dilli LibriSpeech, Almanca, Hollandaca, İspanyolca, Fransızca, İtalyanca, Portekizce ve Lehçe gibi ek dilleri de dahil ederek bunu genişletmektedir. Her dil için hizalanmış transkripsiyonlarla birlikte ses kayıtları sunar. Veri seti, çok dilli TTS sistemleri geliştirmek ve diller arası konuşma sentezi tekniklerini keşfetmek için değerli bir kaynak sağlar.


## VCTK (Voice Cloning Toolkit)

VCTK, metinden konuşmaya araştırma ve geliştirme için özel olarak tasarlanmış bir veri kümesidir. **Çeşitli aksanlara sahip 110 İngilizce konuşmacının ses kayıtlarını içerir**. Her konuşmacı, bir gazeteden, gökkuşağı pasajından ve konuşma aksanı arşivi için kullanılan bir elicitation paragrafından seçilen yaklaşık 400 cümleyi okur. VCTK, TTS modellerini çeşitli sesler ve aksanlarla eğitmek için değerli bir kaynak sunarak daha doğal ve çeşitli konuşma sentezine olanak sağlar.

## Libri-TTS/ LibriTTS-R

Libri-TTS/ LibriTTS-R, Heiga Zen tarafından Google Speech ve Google Brain ekip üyelerinin yardımıyla hazırlanan, 24kHz örnekleme hızında yaklaşık 585 saatlik okunmuş İngilizce konuşmadan oluşan çok konuşmacılı bir İngilizce derlemdir. LibriTTS derlemi TTS araştırmaları için tasarlanmıştır. LibriSpeech derleminin orijinal materyallerinden (LibriVox'tan mp3 ses dosyaları ve Project Gutenberg'den metin dosyaları) türetilmiştir. LibriSpeech derleminden temel farklılıkları aşağıda listelenmiştir:

Ses dosyaları 24kHz örnekleme hızındadır. 
Konuşma cümle sonlarında bölünmüştür. 
Hem orijinal hem de normalleştirilmiş metinler dahil edilmiştir. 
Bağlamsal bilgiler (örneğin, komşu cümleler) çıkarılabilir. 
Önemli arka plan gürültüsü olan ifadeler hariç tutulmuştur. 

TTS için iyi bir veri kümesi oluşturmak kolay bir iş değildir, çünkü böyle bir veri kümesinin birkaç temel özelliğe sahip olması gerekir:

- Çok çeşitli konuşma kalıplarını, aksanları, dilleri ve duyguları kapsayan yüksek kaliteli ve çeşitli kayıtlar. Kayıtlar net olmalı, arka plan gürültüsünden arındırılmalı ve doğal konuşma özellikleri sergilemelidir. 
- Transkripsiyonlar: Her ses kaydına karşılık gelen metin transkripsiyonu eşlik etmelidir. 
- Dilsel içerik çeşitliliği: Veri kümesi, farklı cümle türleri, ifadeler ve kelimeler de dahil olmak üzere çeşitli dilsel içerikler içermelidir. Modelin farklı dilsel bağlamları işleyebilmesini sağlamak için çeşitli konuları, türleri ve etki alanlarını kapsamalıdır. İyi haber şu ki, bir TTS modelini sıfırdan eğitmek zorunda kalmanız pek olası değildir. Bir sonraki bölümde Hub'da bulunan önceden eğitilmiş modelleri inceleyeceğiz.